# Using QCArchive with the OpenFF Toolkit

Here we show how to create OpenFF molecules safely from data in the QCArchive using the CMILES entries. This transformation relies on the `"canonical_isomeric_explicit_hydrogen_mapped_smiles"`.

First load up the client you wish to connect to, in this case, we use the public instance.

In [ ]:
import qcelemental
import qcportal

from openff.toolkit import Molecule

client = qcportal.PortalClient("https://api.qcarchive.molssi.org:443")

print(client.list_datasets_table())

Data in the QCArchive is organized into [datasets](https://molssi.github.io/QCFractal/user_guide/datasets.html#using-datasets). Let's grab a molecule from an optimization dataset

In [ ]:
dataset = client.get_dataset(
    dataset_type="optimization",
    dataset_name="Kinase Inhibitors: WBO Distributions",
)

Take an arbitrary entry from the collection.

In [ ]:
entry = dataset.get_entry(entry_name=dataset.entry_names[-1])

We can view the entry in detail by looking at the dictionary representation.

In [ ]:
entry.dict()

Now we can make a molecule using a few different input options.

In [ ]:
# first make a molecule using this record object
molecule_from_entry = Molecule.from_qcschema(entry)

# we could have also used the dictionary representation of the object
molecule_from_dict = Molecule.from_qcschema(entry.dict())

assert molecule_from_entry == molecule_from_dict

molecule = molecule_from_entry

In [ ]:
# first let's get the initial molecule from the database
initial_molecule = client.get_molecules(entry.initial_molecule.id)
    
# note that this molecule uses an object model from QCArchive, _not_ the toolkit    
print(type(initial_molecule))

# we check that the molecule has been ordered to match the ordering used in the data base
# by printing out the atomic numbers of both objects in order

for atoms in zip(molecule.atoms, initial_molecule.atomic_numbers):
    print(atoms[0].atomic_number, atoms[1])
    assert atoms[0].atomic_number == atoms[1]

# can compare other things, too
print(molecule.to_hill_formula(), initial_molecule.get_molecular_formula())

# QCArchive molecules don't store all information the 
# toolkit needs, like bond orders and formal charges;
# that's why there is a Molecule.from_qcschema() method at all

In [ ]:
# we can also compare the graph representations of the molecules to make sure they are in the same order
import networkx as nx

# make a graph of the initial molecule using newtorkx and the data in the record
initial_network = nx.Graph()
for index, atomic_number in enumerate(initial_molecule.atomic_numbers):
    initial_network.add_node(index, atomic_number=atomic_number)

for bond in initial_molecule.connectivity:
    initial_network.add_edge(*bond[:2])
# now we can use the new isomorphic check to get the atom mapping
isomorphic, atom_map = Molecule.are_isomorphic(
    molecule,
    initial_network,
    return_atom_map=True,
    aromatic_matching=False,
    formal_charge_matching=False,
    bond_order_matching=False,
    bond_stereochemistry_matching=False,
    atom_stereochemistry_matching=False,
)

# we can check if the graph was found to be isomorphic and whether or not the
# atom mappings are in the same order
assert isomorphic

for index1, index2 in atom_map.items():
    assert index1 == index2

Now that we have seen how to make the molecule, lets look at also getting the geometry as currently we have none.

Entries store much information about the molecule, but not the geometry. This has to be pulled down from the server. Since we didn't pass the `client` object to `Molecule.from_qcschema` earlier, the `Molecule` objects we created earlier don't have conformers. (QCArchive's used of "geometry" can be thought of as interchangeable with the toolkit's use of "conformer".)

In [ ]:
# entries don't store geometry, so there are no conformers yet
assert molecule.n_conformers == 0

# if we also want the input geometry for the molecule, we just need to pass the relavent client instance
molecule_with_conformer = Molecule.from_qcschema(entry, client=client)

# check that there is a conformer
assert molecule_with_conformer.n_conformers == 1

In [ ]:
# OpenFF Toolkit `Molecule` objects can be converted back into QCArchive molecules, as long as there are conformer(s)
from openff.toolkit.utils.exceptions import InvalidConformerError

try:
    qc_molecule = molecule.to_qcschema()
except InvalidConformerError:
    qc_molecule = molecule_with_conformer.to_qcschema()

qc_molecule

This transformation unlocks functionality of [QCEngine](https://molssi.github.io/QCEngine/) (computing energies, gradients, hessians, etc. with a variety of different methods).

Here we will try and compute the energy using RDKit (only run this cell if QCEngine is installed.) 

In [ ]:
import qcengine

# set up the RDKit task
rdkit_task = {
    "schema_name": "qcschema_input",
    "schema_version": 2,
    "molecule": qc_molecule,
    "driver": "energy",
    "model": {"method": "uff", "basis": None},
    "keywords": {"scf_type": "df"},
}

# now lets compute the energy using qcengine and RDKit and print the result
result = qcengine.compute(rdkit_task, "rdkit")

In [ ]:
# note the result is in QC units of hartrees
print(result.return_result)